In [73]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pytickersymbols import PyTickerSymbols

pd.options.mode.chained_assignment = None  # default='warn'

In [74]:
# This Cell Block is all the functions for the strat. This is the only cell block that needs to be changed to backtest a different strategy.
def RSI(data, window=14):
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

def strat(ticker, period):   
    # Download stock data from Yahoo Finance
    data = yf.download(ticker, period=period)
    
    # Calculate RSI and add it to the DataFrame
    data['RSI'] = RSI(data)
    
    # Define the buy and sell conditions based on RSI
    data['Signal'] = 0
    data['Signal'][data['RSI'] > 90] = 1   # Buy signal
    data['Signal'][data['RSI'] < 10] = -1  # Sell signal
    
    # Generate positions
    data['Position'] = data['Signal'].shift()

    # Implement strategy
    data['Buy'] = 0
    data['Sell'] = 0

    for i in range(1, len(data)):
        if data['Position'].iloc[i] == 1 and data['Position'].iloc[i-1] != 1:
            data.at[data.index[i], 'Buy'] = data['Open'].iloc[i]
        if data['Position'].iloc[i] == -1 and data['Position'].iloc[i-1] != -1:
            data.at[data.index[i], 'Sell'] = data['Close'].iloc[i]
        if data['Position'].iloc[i] == 1 and (data.index[i] - data.index[i-1]).days >= 7:
            data.at[data.index[i], 'Sell'] = data['Close'].iloc[i]
    
    # Calculate daily returns
    data['Daily Return'] = data['Close'].pct_change()
    # Calculate strategy returns: Position * Daily Return
    data['Strategy Return'] = data['Position'] * data['Daily Return']
    
    # Calculate cumulative returns
    data['Cumulative Market Return'] = (1 + data['Daily Return']).cumprod() - 1
    data['Cumulative Strategy Return'] = (1 + data['Strategy Return']).cumprod() - 1

    # Calculate total strategy return
    total_strategy_return = data['Cumulative Strategy Return'].iloc[-1] * 100
    total_strategy_return = "{:.2f}%".format(total_strategy_return)
    
    return total_strategy_return

In [75]:
#The backtesting function to backtest the strat over different periods
def backtest_strat(ticker):
    backtest_data = {
        'Asset': [ticker],
        'YTD': [strat(ticker, "ytd")],
        'Last 1Y': [strat(ticker, "1y")],
        'Last 6M': [strat(ticker, "6mo")],
        'Last 3M': [strat(ticker, "3mo")],
        'Last 1M': [strat(ticker, "1mo")]    }
    
    # Create a pandas dataframe
    backtest_df = pd.DataFrame(backtest_data)
    
    # Set the first column as the index for clarity
    backtest_df.set_index('Asset', inplace=True)
    return backtest_df

In [76]:
#Creates an empty dataframe to store the backtest data
df =pd.DataFrame({
        'Asset': [],'YTD': [], 'Last 1Y': [], 'Last 6M': [], 'Last 3M': [], 'Last 1M': []
    })

df.set_index('Asset', inplace=True)

In [77]:
#Fetching the tickers for the S&P 500 - [ticker['symbol'] for ticker in sp500_tickers] - ['TSLA', 'NVDA', 'AAPL', 'MSFT', 'MSTR', 'AMZN', 'META', 'INTC']
#This will take like 10mins to execute
stock_data = PyTickerSymbols()
sp500_symbols = stock_data.get_stocks_by_index('S&P 500')
symbols = ['TSLA', 'NVDA', 'AAPL', 'MSFT', 'MSTR', 'AMZN', 'META', 'INTC']

#Running the stocks through the strat and backtesting functions
for symbol in symbols:
    try:
        df = pd.concat([df, backtest_strat(symbol)])
    except IndexError:
        pass


df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,YTD,Last 1Y,Last 6M,Last 3M,Last 1M
Asset,,,,,
TSLA,-6.92%,-10.80%,-5.17%,-1.63%,0.60%
NVDA,12.62%,1.52%,-5.19%,3.57%,-3.13%
AAPL,-1.08%,-0.96%,0.32%,-0.08%,-0.89%
MSFT,-0.22%,-2.52%,-0.49%,-0.61%,0.48%
MSTR,-17.89%,-14.99%,-4.22%,1.33%,-11.20%
AMZN,2.15%,-1.39%,0.68%,-0.82%,-0.43%
META,0.44%,0.77%,-0.52%,-0.64%,0.69%
INTC,-6.88%,3.17%,-2.28%,1.45%,0.95%


In [78]:
#Saves the dataframe to a csv file. Edit the name
df.to_csv('S&P 500 RSI Strat 90 10 - small list.csv', index=True)